# Capstone Project part 2: EDA  #
## Chalmer Lowe

** README ** 

The analysis of computer behavior (both the behavior of individual hosts and the behavior of traffic crossing between hosts) is fundamental to performing cyber security analysis. This EDA session evaluates a portion of the data from a Los Alamos National Labs data set comprising 90 days of host-based data collection and 90 days of network traffic. Due to the extremely large data set, we opted to limit our analysis to a subset of the total data that was collected on day 90. We attempted to include only data from the hosts and the network that fell within the same time slot (from xx:xx to xx:xx). This subset of data was chosen as a means of decreasing the overall data processing time needed.

Our subset of the data is included for use with this notebook.

** EDA comment by instructor**


## Step 0: What are you working on
** Your project title: Who and What: an assessment of the behavior on and between hosts on a network**  
** What are the outcomes of your project: **  
** Data Source:(provide a link to dataset if available)** 
The original data for this project came from XXX:

In [1]:
'''
Time	The start time of the event in epoch time format
Duration	The duration of the event in seconds.
SrcDevice	The device that likely initiated the event.
DstDevice	The receiving device.
Protocol	The protocol number.
SrcPort	The port used by the SrcDevice.
DstPort	The port used by the DstDevice.
SrcPackets	The number of packets the SrcDevice sent during the event.
DstPackets	The number of packets the DstDevice sent during the event.
SrcBytes	The number of bytes the SrcDevice sent during the event.
DstBytes'''

'\nTime\tThe start time of the event in epoch time format\nDuration\tThe duration of the event in seconds.\nSrcDevice\tThe device that likely initiated the event.\nDstDevice\tThe receiving device.\nProtocol\tThe protocol number.\nSrcPort\tThe port used by the SrcDevice.\nDstPort\tThe port used by the DstDevice.\nSrcPackets\tThe number of packets the SrcDevice sent during the event.\nDstPackets\tThe number of packets the DstDevice sent during the event.\nSrcBytes\tThe number of bytes the SrcDevice sent during the event.\nDstBytes'

**Load packages:**

In [2]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
%matplotlib inline

#more packages import as needed

<a id='load_data'></a>

### 1. Load the data

---

Import the csv into a pandas DataFrame.

In [96]:
# As noted in the Readme, above, there was significantly more data available than could be
# conveniently processed and shared as part of this project. For that reason, I did some 
# fast and dirty preprocessing to reduce the number of records for both of the data files.
# The netflow data was reduced to ~3 million records between two time stamps
# The host data was reduced to ~1 million records between roughly the same time stamps
#     Additional processing below will be used to ensure that only records actually 
#     associated with the same time windows will be used in this assessment.


# The first dataset we will look at is the netflow data:
names = ['time',
         'duration',
         'srcdevice',
         'dstdevice',
         'protocol',
         'srcport',
         'dstport',
         'srcpackets',
         'dstpackets',
         'srcbytes',
         'dstbytes',
        ]

NETDATA = 'netflow_day-90_750K'

dfnet = pd.read_csv(NETDATA, names=names)

In [98]:
dfnet.head()  

,time,duration,srcdevice,dstdevice,protocol,srcport,dstport,srcpackets,dstpackets,srcbytes,dstbytes
0,7689600,0,Comp580697,Comp275646,17,Port08290,53,1,0,75,0
1,7689600,0,Comp580697,Comp275646,17,Port81132,53,1,0,75,0
2,7689600,0,Comp580697,Comp275646,17,Port21353,53,1,0,75,0
3,7689600,0,Comp258829,Comp388094,6,Port81881,Port75515,0,3,0,144
4,7689600,0,Comp819564,Comp576031,17,Port10711,514,1,0,245,0


In [97]:
HOSTDATA = 'wls_day-90_250K'

dfhost = pd.read_json(HOSTDATA, lines=True)

In [99]:
dfhost.head()

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ProcessID,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName
0,"{'0': None, '1': None, '2': None, '3': None, '...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': 'Domain001', '1': 'Domain001', '2': 'Dom...","{'0': 4688, '1': 4688, '2': 4688, '3': 4769, '...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': 'Comp883506', '1': 'Comp154941', '2': 'C...","{'0': '0x3e7', '1': '0x3e7', '2': '0x3e7', '3'...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': '0x4db8', '1': '0x3b60', '2': '0x3c8', '...",...,"{'0': '0x4140', '1': '0x36c4', '2': '0xbfc', '...","{'0': 'wermgr.exe', '1': 'Proc857443.exe', '2'...","{'0': None, '1': None, '2': None, '3': 'Active...","{'0': None, '1': None, '2': None, '3': 'Comp23...","{'0': None, '1': None, '2': None, '3': '0x0', ...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': None, '1': None, '2': None, '3': None, '...","{'0': 7689600, '1': 7689600, '2': 7689600, '3'...","{'0': 'Comp883506$', '1': 'Comp154941$', '2': ..."


In [41]:
filtered_data 
# dfjson = pd.read_json('wls_day-90', orient='records', ) #lines=True)

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ProcessID,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName
0,NaN,NaN,Domain001,4688,NaN,Comp883506,0x3e7,NaN,NaN,0x4db8,...,0x4140,wermgr.exe,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp883506$
1,NaN,NaN,Domain001,4688,NaN,Comp154941,0x3e7,NaN,NaN,0x3b60,...,0x36c4,Proc857443.exe,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp154941$
2,NaN,NaN,Domain001,4688,NaN,Comp055724,0x3e7,NaN,NaN,0x3c8,...,0xbfc,taskeng.exe,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp055724$
3,NaN,NaN,Domain001,4769,NaN,ActiveDirectory,NaN,NaN,NaN,NaN,...,NaN,NaN,ActiveDirectory$,Comp230907,0x0,NaN,NaN,NaN,7689600,Comp230907$
4,Kerberos,NaN,Domain001,4624,NaN,ActiveDirectory,0xf014029,3.0,Network,NaN,...,NaN,NaN,NaN,Comp230907,NaN,NaN,NaN,NaN,7689600,Comp230907$
5,NaN,NaN,Domain001,4769,NaN,ActiveDirectory,NaN,NaN,NaN,NaN,...,NaN,NaN,ActiveDirectory$,Comp230907,0x0,NaN,NaN,NaN,7689600,Comp230907$
6,Kerberos,NaN,Domain001,4624,NaN,ActiveDirectory,0xf01403e,3.0,Network,NaN,...,NaN,NaN,NaN,Comp230907,NaN,NaN,NaN,NaN,7689600,Comp230907$
7,NaN,NaN,Domain001,4634,NaN,ActiveDirectory,0xf014029,3.0,Network,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp230907$
8,NaN,NaN,Domain001,4769,NaN,ActiveDirectory,NaN,NaN,NaN,NaN,...,NaN,NaN,ActiveDirectory$,Comp230907,0x0,NaN,NaN,NaN,7689600,Comp230907$
9,NaN,NaN,Domain001,4634,NaN,ActiveDirectory,0xf007f26,3.0,Network,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp734626$


In [49]:
print('total time slice:', 7691436 - 7689600, 'seconds')

total time slice: 1836 seconds


### 2. Describe the basic format of the data and the columns

---

Use the `.head()` function (and optionally pass in an integer for the number of rows you want to see) to examine what the loaded data looks like. This is a good initial step to get a feel for what is in the csv and what problems may be present.

The `.dtypes` attribute tells you the data type for each of your columns.

Describe what you see

In [38]:
df.head(10)

,time,duration,srcdevice,dstdevice,protocol,srcport,dstport,srcpackets,dstpackets,srcbytes,dstbytes
0,7689600,0,Comp580697,Comp275646,17,Port08290,53,1,0,75,0
1,7689600,0,Comp580697,Comp275646,17,Port81132,53,1,0,75,0
2,7689600,0,Comp580697,Comp275646,17,Port21353,53,1,0,75,0
3,7689600,0,Comp258829,Comp388094,6,Port81881,Port75515,0,3,0,144
4,7689600,0,Comp819564,Comp576031,17,Port10711,514,1,0,245,0
5,7689600,0,Comp518262,Comp576843,6,Port91388,80,5,0,400,0
6,7689600,0,Comp030371,Comp576843,6,Port37528,80,5,0,398,0
7,7689600,0,Comp509586,Comp275646,17,Port20133,53,1,0,72,0
8,7689600,0,Comp509586,ActiveDirectory,6,Port02703,389,10,0,2759,0
9,7689600,0,Comp509586,ActiveDirectory,6,Port55089,389,8,0,2322,0


In [39]:
df.dtypes

time           int64
duration       int64
srcdevice     object
dstdevice     object
protocol       int64
srcport       object
dstport       object
srcpackets     int64
dstpackets     int64
srcbytes       int64
dstbytes       int64
dtype: object

---

Use the `.info()` function  to see how many rows and data types you got. This will also tell you how many rows of each column have content

Describe what you see

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
time          100000 non-null int64
duration      100000 non-null int64
srcdevice     100000 non-null object
dstdevice     100000 non-null object
protocol      100000 non-null int64
srcport       100000 non-null object
dstport       100000 non-null object
srcpackets    100000 non-null int64
dstpackets    100000 non-null int64
srcbytes      100000 non-null int64
dstbytes      100000 non-null int64
dtypes: int64(7), object(4)
memory usage: 8.4+ MB


In [51]:
df.describe(include=np.number)

,time,duration,protocol,srcpackets,dstpackets,srcbytes,dstbytes
count,1.000000e+05,100000.000000,100000.000000,1.000000e+05,1.000000e+05,1.000000e+05,1.000000e+05
mean,7.689621e+06,320.864420,9.288610,2.315531e+02,4.016728e+02,3.665285e+04,1.817803e+05
std,1.227961e+01,3337.662621,5.038413,3.895550e+04,7.834661e+04,3.491970e+06,2.032624e+07
min,7.689600e+06,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.689611e+06,0.000000,6.000000,1.000000e+00,0.000000e+00,7.600000e+01,0.000000e+00
50%,7.689622e+06,1.000000,6.000000,4.000000e+00,1.000000e+00,2.770000e+02,1.060000e+02
75%,7.689632e+06,2.000000,17.000000,6.000000e+00,5.000000e+00,9.640000e+02,8.050000e+02
max,7.689642e+06,85892.000000,17.000000,1.189244e+07,2.425934e+07,7.673681e+08,3.144187e+09


In [52]:
df.describe(include=object)

,srcdevice,dstdevice,srcport,dstport
count,100000,100000,100000,100000
unique,7611,2618,41583,2412
top,Comp965575,Comp275646,123,53
freq,8446,15133,451,16115


<a id='drop'></a>

### 2. Drop unwanted columns

---
You may want to focus on selected columns as your input features. You can drop those unwanted columns.

The `.drop()` built-in function can be used to get rid of a column. When removing a column, we need to specify `axis=1` to the function.

For the record, the `.index` attribute holds the row indices. This is the the sister attribute to the `.columns` attribute that we work with more often.



In [ ]:
# Remove the unneccesary column:


### 3. Clean corrupted columns (if there is any)

---

Did you notice when you examined the `dtypes` attribute that some of the columns were of type "object", indicating that they were string. For some case, these columns should be numeric instead.

It is pretty common to have numeric columns represented as strings in your data if some of the observations are corrupted. It is important to always check the data types of your columns.

In [ ]:
# Use this to inspect what content you have in columns marked 'object'
# df['column_name'].values 

In [ ]:
# If you find a pattern of corrupted content in the column marked as object type, 
# you may replace it with some meaningful numerical content
# df['column_name'] = [ float(x.replace('old','new')) for x in df['column_name']]


In [ ]:
#check to see if the corrupted columns got corrected and become a numerical columns
#Look into df.info() again


### 4. Determine how many observations are missing

---

When we start to build models with data, null values in observations are (almost) never allowed. It is important to always see how many observations are missing and for which columns.

A handy way to look at how many null values there are per column with pandas is:

```python
boston.isnull().sum()
```

The `.isull()` built-in function will convert the columns to boolean `True` and `False` values (returning a new dataframe) where null values are indicated by `True`. 

The `.sum()` function tacked on to the back of that will then sum these boolean columns, and the total number of null values per column will be returned.

In [ ]:
# How many columns have how null values?

**Drop the null values.** 

In this case, lets keep it simple and just drop the rows from the dataset that contain null values. If a column has a ton of null values it often makes more sense to drop the column entirely instead of the rows with null values. In this case, we will just drop the rows.

The `.dropna()` function will drop any rows that have _**ANY**_ null values for you.  Use this carefully as you could drop many more rows than expected.

** Instead of dropping null values, you may want to impute them (replace with the mean(numeric type) or most frequently occured item(object type) **

In [ ]:
# use df.shape to check the size before dropping anything

In [ ]:
# Now drop the rows with null values
# df.dropna(inplace=True)
# check the new size after dropping by df.shape

### 5. Make the column names more descriptive

---

Often it is annoying to have to memorize what the codes mean for columns, or reference the codebook whenever I want to know the meaning of a variable. It often makes sense to rename columns that are not descriptive.

There is more than one way to do this, but one easy way is to use the `.rename()` function.


In [ ]:
# Look at the original column names
# df.columns

There two popular methods to rename dataframe columns.
1. Using a _dictionary substitution_, which is very useful if you only want to rename a few of the columns. This method uses the `.rename()` function.
2. Using a _list replacement_, which is quicker than writing out a dictionary, but requires a full list of names.

In [ ]:
# Dictionary Method
# new_column={
#         'column1_old':'column1_new',
#         'column2_old':'column2_new',
#         #more here,
#         'columnlast_old':'columnlast_new'    
# }
# df.rename(columns=new_column, inplace =True)
# df.head(2)

In [ ]:
# List Replacement Method
# new_names = ['column1_new','column2_new',
#              #more here
#             'columnlast_new']
# df.columns=new_names
# df.head(2)

<a id='describe'></a>

### 6. Describe the summary statistics for the columns

---

The `.describe()` function gives summary statistics for each of your variables. What are some, if any, oddities you notice about the variables based on this output?

In [53]:
# A:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
time,100000.0,7.689621e+06,1.227961e+01,7689600.0,7689611.0,7689622.0,7689632.0,7.689642e+06
duration,100000.0,3.208644e+02,3.337663e+03,0.0,0.0,1.0,2.0,8.589200e+04
protocol,100000.0,9.288610e+00,5.038413e+00,1.0,6.0,6.0,17.0,1.700000e+01
srcpackets,100000.0,2.315531e+02,3.895550e+04,0.0,1.0,4.0,6.0,1.189244e+07
dstpackets,100000.0,4.016728e+02,7.834661e+04,0.0,0.0,1.0,5.0,2.425934e+07
srcbytes,100000.0,3.665285e+04,3.491970e+06,0.0,76.0,277.0,964.0,7.673681e+08
dstbytes,100000.0,1.817803e+05,2.032624e+07,0.0,0.0,106.0,805.0,3.144187e+09


### 7. Plot variables with potential outliers using boxplots.

---
You can use the seaborn package to plot boxplots of the variables we have identified as potentially having outliers.

Some notes on seaborn's boxplot keyword argument options:

    orient: can be 'v' or 'h' for vertical and horizontal, respectively
    fliersize: the size of the outlier points (pixels I think)
    linewidth: the width of line outlining the boxplot
    notch: show the confidence interval for the median (calculated by seaborn/plt.boxplot)
    saturation: saturate the colors to an extent
** What did you see and conclude out these boxplots?

### 8.Other relevants plots 

---
What can you infer out of these additonal plots as related to your problem?

In [ ]:
# A:


<a id='standardization'></a>

### 9. Standardizing variables

---

Rescaling variables is very common, and sometimes essential. For example, when we get to regularization of models the rescaling procedure becomes a requirement before fitting the model.

Here we'll rescale the variables using a procedure called "standardization", which forces the distribution of each variable to have a mean of 0 and a standard deviation of 1.

Standardization is not complicated:

    standardized_variable = (variable - mean_of_variable) / std_dev_of_variable
    
Note: Nothing else is changed about the distribution of the variable. It doesn't become normally distributed.


Also print out the mean and standard deviation of the original variable.

In [1]:
# select the columns out of your dataframe to be standardized (usually it will be your input features)
# X= df[['column1','column2', 'more if needed']]
#check its mean and standard deviation before standardizing
# X.mean()
# X.std()


In [ ]:
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)
#check its mean and standard deviation after standardizing
#X_standardized.mean()
# X_standardized.std()
#What difference did you see? Describe here


### 10. Correlation matrices

---

A great way to easily get a feel for linear relationships between your variables is with a correlation matrix.



####  Correlation


A much more common metric, and one directly calculable from the covariance, is the correlation.

Again, let $X$ and $Y$ be our two variables, with covariance $cov(X, Y)$ that we calculated above:

### $$ \text{pearson correlation}\;r = cor(X, Y) =\frac{cov(X, Y)}{std(X)std(Y)}$$



#### The correlation matrix

We can see the correlation between all the numeric variables in our dataset by using pandas DataFrame's built in `.corr()` function. Use it below on the boston dataset.

It is very useful to get a feel for what is related and what is not, which can help you decide what is worth investigating further (though with a lot of variables, the matrix can be a bit overwhelming...)

In [ ]:
# df.corr()

**Seaborn also has a great way of showing this to us visually, if colors stick out to you more than decimol values.**

In [ ]:
# A:
# sns.heatmap(df.corr())

## more EDA related to your dataset here:

## Describe has this EDA helped you to understand and get ready for your next step in your capstone project. How?

# Backup code for processing the original data:

In [106]:
# Read in data from the original file and save it to a new, smaller file.
# NOTE: cell is saved in RAW format to prevent execution.

import time

max_records = 1000
HOSTDATA = 'wls_day-90'

df = pd.read_json(HOSTDATA, lines=True, chunksize=max_records)
filtered_data = pd.DataFrame() # Initialize the dataframe

start = time.time()
try:
    for index, df_chunk in enumerate(df):
        filtered_data = pd.concat([filtered_data, df_chunk])
        # print('success')
        if index > 215:
            break
        
except OSError:
    print ('\nEPIC FAIL')
    print(index)

filtered_data.to_json('wls_day-90_250K', lines=True, orient='records')

print('Total processing time:', (time.time() - start)/60)  

Total processing time: 0.5160190979639689


In [115]:
len(filtered_data)

217000

In [107]:
filtered_data.head(2)

,AuthenticationPackage,Destination,DomainName,EventID,FailureReason,LogHost,LogonID,LogonType,LogonTypeDescription,ParentProcessID,...,ProcessID,ProcessName,ServiceName,Source,Status,SubjectDomainName,SubjectLogonID,SubjectUserName,Time,UserName
0,NaN,NaN,Domain001,4688,NaN,Comp883506,0x3e7,NaN,NaN,0x4db8,...,0x4140,wermgr.exe,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp883506$
1,NaN,NaN,Domain001,4688,NaN,Comp154941,0x3e7,NaN,NaN,0x3b60,...,0x36c4,Proc857443.exe,NaN,NaN,NaN,NaN,NaN,NaN,7689600,Comp154941$


In [108]:
print('total time slice:', filtered_data.Time.max() - filtered_data.Time.min(), 'seconds')
print(filtered_data.Time.max(), filtered_data.Time.min())

total time slice: 368 seconds
7689968 7689600


In [113]:
names = ['time',
         'duration',
         'srcdevice',
         'dstdevice',
         'protocol',
         'srcport',
         'dstport',
         'srcpackets',
         'dstpackets',
         'srcbytes',
         'dstbytes',
        ]

dfnet = pd.read_csv('netflow_day-90', names=names, nrows=680000)
dfnet.to_csv('netflow_day-90_x')

In [114]:
print('total time slice:', dfnet.time.max() - dfnet.time.min(), 'seconds')
print(dfnet.time.max(), dfnet.time.min())

total time slice: 371 seconds
7689971 7689600


In [94]:
len(dfnet)

750000